In [51]:
from nyse_dates_prds import NYSE_holidays2
from RealTimeNN import *
from dateutil import rrule 
import matplotlib.patches as mpatches
import matplotlib.pyplot as plt
import cPickle as pickle
import pandas as pd
import numpy as np
import scipy.linalg.blas
import datetime
import random
import scipy
import time

print "LOADED"

LOADED


In [2]:
def create_neural_network(neural_network, delay_input=[0], delay_internal=[], delay_output=[]):
    """ Create Neural Network
    Example: 
      network         = [2,3,4,1] network w/ 2 inputs, 2 hidden layers w/ 3 and 4 neurons, 
                                  and 1 linear output layer                             
      delay_input     = [0,1,5] will use inputs of timestep t, t-1 and t-5
      delay_internal  = [1,2,5] adds recurrent connection for output of each layer 
                                in timestep t-1, t-2 and t-5
      delay_output    = [1,3,4] adds recurrent connection for output of each layer in 
                                timestep t-1, t-3 and t-5
    
    Args:
        network:        structure of the neural network [I HL1 HL2 ... HLN OL]
                        number of layers is the length of the list-1
                        number neurons in each layer is the given number
    
        delay_input:    Time delays for NN inputs. 
                        To use only the input of timestep t dIn = [0]
            
        delay_internal: Time delays for recurrent internal connections of NN.
                        dIntern has to be greater than zero (layer output at timestep t-x)!
                        if non-empty list given, recurrent connection from every layer 
                        to itself and every layer before is added
            
        delay_output:   Time delays for recurrent connections of output to first hidden 
                        layer. dOut has to be greater than zero (output at timestep t-x)!
                        if non-empty list given, recurrent connection from NN output to first 
                        hidden layer is added 
    
    Returns:
        
        net: untrained neural network 
    """
    
    network                = {}          
    
    network['delay']       = {'input'    :delay_input, 
                              'internal' :delay_internal, 
                              'output'   :delay_output}   
    
    #Structure
    network['network']     = neural_network  
    
    network['num_layers']  = len(neural_network) - 1  
    
    #structure without inputs
    network['layers']      = neural_network[1:] 
    
    #maximum time delay
    network['max_delay']   = max( max(delay_input, delay_internal, delay_output) )
    
    #initialize random weight vector and specify sets
    network                = create_weight_vector(network)    
    
    #weight vector used for calculation
    network['weight_vect'] = network['w0'].copy()
    
    #number of weights
    network['num_weights'] = len(network['w0'])           
    
    return network

print "LOADED"

LOADED


In [3]:
def create_weight_vector(network):
    """
    Creates random weight vector of NN and defines sets needed for derivative calculation
    
    Returns: 
        neural network
    """
    
    num_layers_network  = network['num_layers']   
    layers              = network['layers'] 
    delay               = network['delay'] 
    inputs              = network['network'][0]
    
    # Input layers or layers w/ internal delay > 0
    input_layers     = [] 
    # Output of layer used for cost func calc or added to input layer w/ delay > 1
    output_layers    = []  
    
    # Connection weight matrix layer m -> layer l w/ delay d
    connection_weight_matrices  = {}  
    bias                        = {}
    input_weight_matrices       = {}
    layers_bkwd_connect_layerM  = {} 
    layers_fwd_connect_layerM   = {} 
    delay_layerM_toL            = {}   
    input_layersU_connect_to    = {}            
    output_layersX_connect_to   = {}
            
    '''Inputs'''
    inputs_connect_layer1      = {} 
    inputs_connect_layer1[1]   = [1]           
    delay_input_layer1         = {}
    delay_input_layer1[1, 1]   = delay['input']
  
    for x in delay_input_layer1[1, 1]:
        # Input-weight matrix set to random values [-0.5,0.5]
        input_weight_matrices[1, 1, x] = np.random.rand(layers[0], inputs) - 0.5  
    
    # First layer is input layer
    input_layers.append(1)  
    
    '''Internal Connection Weight Matrices'''
    for m in range(1, num_layers_network + 1):
        layers_bkwd_connect_layerM[m] = []     
        layers_fwd_connect_layerM[m]  = []
            
        # Forward connects
        if m > 1:
            l = m - 1
            # No delay for forward connects
            delay_layerM_toL[m, l]              = [0]                                            
            connection_weight_matrices[m, l, 0] = np.random.rand(layers[m - 1], 
                                                                 layers[l - 1]) - 0.5 
            
            layers_bkwd_connect_layerM[l].append(m)  
            layers_fwd_connect_layerM[m].append(l)      
    
        # Recursive connects
        for l in range(m, num_layers_network + 1):
            if (m == 1) and (l == num_layers_network):            
                # Delays from output to first layer
                delay_layerM_toL[m, l] = delay['output']          
            else:
                # Internal delays
                delay_layerM_toL[m, l] = delay['internal']       
                
            # All delays for connect l->m    
            for d in delay_layerM_toL[m, l]:                                                     
                connection_weight_matrices[m, l, d] = np.random.rand(layers[m - 1], 
                                                                     layers[l - 1]) - 0.5
                
                # Add if haven't yet
                if (l not in layers_fwd_connect_layerM[m]): 
                    layers_fwd_connect_layerM[m].append(l)  
                    
                # If recurrent connect
                if (l >= m) and(d > 0): 
                    if (m not in input_layers):  
                        input_layers.append(m) 
                        
                    if (l not in output_layers): 
                        output_layers.append(l)
        
        # Create bias vect for layer m
        bias[m] = np.random.rand(layers[m - 1]) - 0.5  
    
    if num_layers_network not in output_layers:
        output_layers.append(num_layers_network)
        
    for u in output_layers:
        input_layersU_connect_to[u] = []
        
        for x in input_layers: 
            # If input layer in lfwd[x] 
            #  and connect x -> u has delay > 0 
            #  and x not yet in inputlayersUconnto[u]
            if  (u in layers_fwd_connect_layerM[x]) and \
                (np.any(np.array(delay_layerM_toL[x, u]) > 0)) and \
                (x not in input_layersU_connect_to[u]):
                
                input_layersU_connect_to[u].append(x)
                
    for x in range(1, num_layers_network + 1):
        output_layersX_connect_to[x] = []
        
        for u in output_layers:
            try:
                # If connect u -> x has delay > 0
                if np.any(np.array(delay_layerM_toL[x, u]) > 0): 
                    output_layersX_connect_to[x].append(u)
            
            except KeyError:
                pass
            
    #Add to network
    network['output_layers']               = output_layers
    network['input_layers']                = input_layers
    network['delay_layerM_toL']            = delay_layerM_toL
    network['delay_input_layer1']          = delay_input_layer1
    network['layers_bkwd_connect_layerM']  = layers_bkwd_connect_layerM
    network['layers_fwd_connect_layerM']   = layers_fwd_connect_layerM
    network['inputs_connect_layer1']       = inputs_connect_layer1
    network['input_layersU_connect_to']    = input_layersU_connect_to
    network['output_layersX_connect_to']   = output_layersX_connect_to
    
    network['w0'] = convert_matrices_to_vector(network, 
                                               input_weight_matrices, 
                                               connection_weight_matrices, 
                                               bias)
    return network

print "LOADED"

LOADED


In [4]:
def convert_matrices_to_vector(network, input_weight_matrices, connection_weight_matrices, bias):
    """
    Converts input weight matrices(IW), connection weight matrices(LW) and bias 
    vectors(b) to weight vector(w)
    """
    
    delay_layerM_toL           = network['delay_layerM_toL']    
    delay_input_layer1         = network['delay_input_layer1']    
    inputs_connect_layer1      = network['inputs_connect_layer1']     
    layers_fwd_connect_layerM  = network['layers_fwd_connect_layerM']   
    num_layers_network         = network['num_layers']     
    weight_vect                = np.array([])
    
    # Input weights
    for m in range(1, num_layers_network + 1): 
        if m == 1:
            for i in inputs_connect_layer1[m]:
                for d in delay_input_layer1[m, i]:
                    weight_vect = np.append(weight_vect, 
                                            input_weight_matrices[m, i, d].flatten('F'))
                    
        # Internal connect weights
        for l in layers_fwd_connect_layerM[m]:
            for d in delay_layerM_toL[m, l]:
                weight_vect = np.append(weight_vect, 
                                        connection_weight_matrices[m, l, d].flatten('F'))
                
        # Bias weights
        weight_vect = np.append(weight_vect, bias[m])
    
    return weight_vect

print "LOADED"

LOADED


In [7]:
def RTRL(network, data):
    """ 
    Jacobian Matrix    == derivatives of error with respect to weight vector
    Mean Squared Error == MSE of network compared to training data
    Error Vector       == difference of network output and target data
    """
    
    #data == training data
    network_inputs               = data['inputs']      
    network_outputs              = data['outputs']      
    layer_outputs                = data['layer_outputs']
    num_prev_data_pts            = data['q0'] 
    
    delay_layerM_toL             = network['delay_layerM_toL']      
    delay_input_layer1           = network['delay_input_layer1']
    
    inputs_connect_layer1        = network['inputs_connect_layer1']       
    layers_fwd_connect_layerM    = network['layers_fwd_connect_layerM']     
    layers_bkwd_connect_layerM   = network['layers_bkwd_connect_layerM']     
    num_layers_network           = network['num_layers']   
    
    inputs                       = network['network'][0]   
    outputs                      = network['network'][-1]
    
    layers                       = network['layers']  
    input_layers                 = network['output_layers']       
    output_layers                = network['input_layers']       
    output_layersX_connect_to    = network['output_layersX_connect_to']   
    
    input_weight_matrices, connection_weight_matrices, bias = convert_vector_to_matrices(network)
    
    # 1. Calc network output
    network_out, sum_output_layers, layer_outputs = get_network_output(
        network_inputs, 
        network, 
        input_weight_matrices,                                     
        connection_weight_matrices, 
        bias, 
        layer_outputs     = layer_outputs, 
        num_prev_data_pts = num_prev_data_pts)
    
    # 2. Calc cost func
    error_matrix       = network_outputs - network_out                                             
    error_vector       = np.reshape(error_matrix, (1, np.size(error_matrix)), order = 'F')[0]
    mean_squared_error = np.dot(error_vector, error_vector.transpose())/float(len(error_vector))
    
    # 3. Backpropagation RTRL
    num_of_input_datapts            = network_inputs.shape[1]                       
    num_of_datapts_without_old_data = num_of_input_datapts - num_prev_data_pts 
    
    
    deriv_layer_outputsOfU_respect_bias_vect                     = {} 
    deriv_layer_outputsOfU_respect_input_weight_matrices         = {} 
    deriv_layer_outputsOfU_respect_connectection_weight_matrices = {}
    deriv_layer_outputs_respect_weight_vect                      = {} 
    sensitivity_matrix                                           = {} 
    layersM_with_existing_sensitivity_matrix                     = {} 
    input_layersX_with_existing_sensitivity_matrix               = {} 
                                                                        
    # Init
    jacobian_matrix = np.zeros((num_of_datapts_without_old_data * layers[-1], 
                                network['num_weights']))
    
    for q in range(1, num_prev_data_pts + 1):
        for u in input_layers:
            deriv_layer_outputsOfU_respect_connectection_weight_matrices[q, u] = np.zeros(
                (layers[u - 1], 
                network['num_weights']))
    
    #########
    max_delay  = network['max_delay']
    max_layers = len(layers)
    #########
    
    # Begin RTRL
    for q in range(num_prev_data_pts + 1, num_of_input_datapts + 1):
        # Init, set needed for calculating sensitivities
        input_layers_ = [] 
        for u in input_layers:
            layersM_with_existing_sensitivity_matrix[u]       = []
            input_layersX_with_existing_sensitivity_matrix[u] = []
            deriv_layer_outputs_respect_weight_vect[q, u]     = 0
        
        # Calc sensitivity matrices, decrement m in backprop order
        for m in range(num_layers_network, 0, -1): 
            for u in input_layers_:
                # Sensitivity Matrix layer u -> m
                sensitivity_matrix[q, u, m] = 0 
                
                for l in layers_bkwd_connect_layerM[m]:
                    #recursive calculation of Sensitivity Matrix layer u -> m
                    sensitivity_matrix[q, u, m] += np.dot(
                        np.dot(sensitivity_matrix[q, u, l], connection_weight_matrices[l, m, 0]),
                        np.diag(1 - (np.tanh(sum_output_layers[q, m])) ** 2)) 
                      
                if m not in layersM_with_existing_sensitivity_matrix[u]:
                    layersM_with_existing_sensitivity_matrix[u].append(m) 
                    
                    if m in output_layers:
                        input_layersX_with_existing_sensitivity_matrix[u].append(m)
                        
            if m in input_layers:
                # Output layer is linear, no transfer function
                if m == num_layers_network: 
                    sensitivity_matrix[q, m, m] = np.diag(np.ones(outputs)) 
                else:
                    sensitivity_matrix[q, m, m] = np.diag(1 - (np.tanh(sum_output_layers[q, m])) 
                                                                              ** 2)
                
                # Add m to U'
                input_layers_.append(m) 
                layersM_with_existing_sensitivity_matrix[m].append(m)
                
                if m in output_layers:
                    input_layersX_with_existing_sensitivity_matrix[m].append(m)
          
        '''Calc derivs, static deriv calc'''
        for u in sorted(input_layers): 
            # Static deriv vector: explicit deriv layer outputs w/ respect to weight vect
            deriv_layer_outputs_respect_weight_vect_ = np.empty((layers[u - 1], 0))
            
            # Input weights
            for m in range(1, num_layers_network + 1): 
                if m == 1:
                    for i in inputs_connect_layer1[m]:
                        for d in delay_input_layer1[m, i]:
                            # If no sensivity matrix exists or d >= q: deriv is zero
                            if ((q, u, m) not in sensitivity_matrix.keys()) or (d >= q): 
                                deriv_layer_outputsOfU_respect_input_weight_matrices[m, i, d] = \
                                    np.kron(network_inputs[:, q - d - 1].transpose(), 
                                    np.zeros((layers[u - 1], layers[m - 1])))
                            else: 
                                deriv_layer_outputsOfU_respect_input_weight_matrices[m,i,d] = \
                                    np.kron(network_inputs[:, q - d - 1].transpose(), 
                                    sensitivity_matrix[q, u, m])

                            # Append to static deriv vect
                            deriv_layer_outputs_respect_weight_vect_ = np.append(
                                deriv_layer_outputs_respect_weight_vect_, 
                                deriv_layer_outputsOfU_respect_input_weight_matrices[m, i, d], 
                                1) 
        
                # Connect weights
                for l in layers_fwd_connect_layerM[m]:
                    for d in delay_layerM_toL[m, l]:
                        # If no sensivity matrix exists or d >= q: deriv is zero
                        if ((q, u, m) not in sensitivity_matrix.keys()) or (d >= q): 
                            deriv_layer_outputsOfU_respect_connectection_weight_matrices[m,l,d]=\
                                np.kron(layer_outputs[q, l].transpose(), np.zeros((
                                            layers[u - 1], layers[m - 1])))
                        else:
                            deriv_layer_outputsOfU_respect_connectection_weight_matrices[m,l,d]=\
                                np.kron(layer_outputs[q - d, l].transpose(), 
                                        sensitivity_matrix[q, u, m]) 

                        # Append to static deriv vect
                        deriv_layer_outputs_respect_weight_vect_ = np.append(
                            deriv_layer_outputs_respect_weight_vect_, 
                            deriv_layer_outputsOfU_respect_connectection_weight_matrices[m,l,d], 
                            1) 
                        
                # Bias weights
                if ((q, u, m) not in sensitivity_matrix.keys()):
                    # Deriv is zero
                    deriv_layer_outputsOfU_respect_bias_vect[m] = np.zeros((layers[u - 1], 
                                                                            layers[m - 1])) 
                else:
                    deriv_layer_outputsOfU_respect_bias_vect[m] = sensitivity_matrix[q, u, m] 

                # Append to static deriv vect
                deriv_layer_outputs_respect_weight_vect_ = np.append(
                                        deriv_layer_outputs_respect_weight_vect_, 
                                        deriv_layer_outputsOfU_respect_bias_vect[m], 
                                        1) 
            
            '''Dynamic deriv calc'''
            dyn_deriv_sum_allX = 0
            for x in input_layersX_with_existing_sensitivity_matrix[u]:
                # Sum of all u_
                sum_u_ = 0 
                
                for u_ in output_layersX_connect_to[x]:
                    # Sum of all d
                    sum_d = 0 
                    
                    for d in delay_layerM_toL[x, u_]:
                        # Delays > 0 and < q
                        if (q - d > 0) and (d > 0): 
                            sum_d += np.dot(connection_weight_matrices[x, u_, d], 
                                            deriv_layer_outputs_respect_weight_vect[q - d, u_])

                    sum_u_ += sum_d

                if sum_u_ is not 0:
                    # Sum up dynamic deriv
                    dyn_deriv_sum_allX += np.dot(sensitivity_matrix[q, u, x], sum_u_) 
            
            # Static + dynamic deriv, total deriv output layer u with respect to w
            deriv_layer_outputs_respect_weight_vect[q, u] = \
                    deriv_layer_outputs_respect_weight_vect_ + dyn_deriv_sum_allX 
        # Jacobian matrix
        jacobian_matrix[range(((q - num_prev_data_pts) - 1) * outputs, 
                              (q - num_prev_data_pts) * outputs), :] = \
            -deriv_layer_outputs_respect_weight_vect[q, num_layers_network]

        ############!!!!#########!!!#################!@!@!@!@!#####
        if q > max_delay:
            new_dA_dw = {}
            for dd in range(max_delay):
                for ll in xrange(1,max_layers+1):
                    new_dA_dw[(q-dd,ll)] = deriv_layer_outputs_respect_weight_vect[(q-dd,ll)]
            deriv_layer_outputs_respect_weight_vect = new_dA_dw
            sensitivity_matrix = {}
        ############!!!!#########!!!#################!@!@!@!@!#####

    return jacobian_matrix, mean_squared_error, error_vector

print "LOADED"

LOADED


In [9]:
def train_LM(nn_inputs, 
             nn_outputs, 
             net, 
             iteration_max = 10, 
             MSE_stop      = 1e-6, 
             damp_factor   = 3.0, 
             damp_const    = 10.0, 
             verbose       = False):
    """
    Levenberg-Marquardt(LM) algorithm
      - Least-squares estimation of nonlinear parameters
      
    Args:
        iteration_max:  max # of iterations
        MSE_stop:       termination error, training stops when MSE <= MSE_stop
        damp_const:     constant to adapt damping factor of LM
        damp_factor:    damping factor of LM
    
    Returns:
        net:            trained Neural Network 
    """
    data, net = prepare_data(nn_inputs, nn_outputs, net)
    
    # Calc for first iteration
    Jacobian, Mean_squared_error, error_vect = RTRL(net, data)
    
    # Vect for error history
    iteration               = 0
    ErrorHistory            = np.zeros(iteration_max + 1) 
    ErrorHistory[iteration] = Mean_squared_error
    
    if verbose:
        print('Iteration: ',    iteration, 
              'Error: ',        Mean_squared_error, 
              'scale factor: ', damp_factor)

    # Run loop until either max iterations or MSE_stop reached
    while True:
        #####
        seconds, minutes, hours = 60, 60, 2
        t_end = time.time() + (seconds * minutes * hours)
        
        JTJ = scipy.linalg.blas.dgemm(alpha=1.0, a=Jacobian.T, b=Jacobian.T, trans_b=True)
        #####
        weight_vect = net['weight_vect']
        
        # Repeat until optimizing step successful
        while True:
            gradient = np.dot(Jacobian.transpose(), error_vect)

            # Calc scaled inverse Hessian
            try:
                #####
                scaled_inv_hessian = scipy.linalg.inv(JTJ + damp_factor * 
                                                      np.eye(net['num_weights'])) 
                #####
            except scipy.linalg.LinAlgError:
                # Not invertible, do small step in gradient direction
                weight_delta = 1.0 / 1e10 * gradient
            else:
                # Calc weight modification
                weight_delta = np.dot(-scaled_inv_hessian, gradient)
            
            # New weight vect
            net['weight_vect'] = weight_vect + weight_delta  
            new_mean_squared_error = calc_error(net, data)
            
            # If optimization step successful, adapt scale factor, then go next iteration
            if new_mean_squared_error < Mean_squared_error:
                damp_factor = damp_factor / damp_const 
                break
            elif time.time() > t_end:
                return net
            else:
                damp_factor = damp_factor * damp_const
        
        # Calc for next iteration
        Jacobian, Mean_squared_error, error_vect = RTRL(net, data)
        iteration += 1
        ErrorHistory[iteration] = Mean_squared_error
        
        if verbose:
            print('Iteration: ',    iteration,
                  'Error: ',        Mean_squared_error,
                  'scale factor: ', damp_factor)

        # Check if termination condition hit
        if iteration >= iteration_max:
            print('Max # of iterations reached')
            break
        elif Mean_squared_error <= MSE_stop:
            print('Termination error reached')
            break

    net['ErrorHistory'] = ErrorHistory[:iteration]
    return net

print "DONE"

DONE


In [10]:
def calc_error(network, data):
    """
    Returns:
        MAE of network compared to training data
    """
    
    network_inputs    = data['inputs']
    network_outputs   = data['outputs'] 
    layer_outputs     = data['layer_outputs']
    num_prev_data_pts = data['q0'] 
    
    input_weight_matrices, connection_weight_matrices, bias = convert_vector_to_matrices(network)
    
    network_out, sum_output_layers, layer_outputs = get_network_output(
        network_inputs, 
        network, 
        input_weight_matrices,                                     
        connection_weight_matrices, 
        bias, 
        layer_outputs     = layer_outputs, 
        num_prev_data_pts = num_prev_data_pts)

    # Outputs_delta = error matrix
    outputs_delta       = network_outputs - network_out 
    error_vect          = np.reshape(outputs_delta, (1, np.size(outputs_delta)), order='F')[0]
    Mean_squared_error  = np.dot(error_vect, error_vect.transpose())/float(len(error_vect))

    #return Mean_squared_error
    return Mean_squared_error

print "LOADED"

LOADED


In [2]:
def create_learntest(num, key, out_num, net=False):
    """
    Retrieve our dataframes with all of our indicators for our 6 helper indexes/funds that 
    help in the prediction process for a certain company we choose to predict on. The 6 
    indexes are the Dow Jones Index, the S&P 500 Index, the Nasdaq Composite, 
    United States Oil Fund, the SPDR S&P 500 ETF, and SPDR Gold Shares. 
    
    We take these dataframes, and concatenate to our stock we will predict for,
    calling it with a stock key that we feed in through a parameter. 
    
    We then adjust for missing dates that many of the 6 helper stocks lack but that many
    stocks do have, removing those certain dates from our data. We then take the list of 
    indicators we are going to use by loading a list with all of the 474 names of 
    indicators. There are over 2000 possible indicators, we chose only 474 for memory and
    computation costs of using more than that. 
    
    You can change the indicators as you like, just as long as it's a list of indicator 
    names as strings. We get our 474 indicator dataframe then set our output as our stock
    choices return values at 1 minute intervals. 
    
    We then set up our learn/test lists which have to be lists due to better efficiency
    using numpy rather than pandas dataframes.
    
    Return the inputs, outputs, test inputs, test outputs, and the neural net if we already
    have one set up.
    """
    # Grab our pickled files from our NewBase/STOCKSYMBOL/ directories
    # the num is referring to one of fifteen parts to each company indicator dataframe
    opp       = open('NewBase/^GSPC/^GSPC_df'+str(num)+'.pickle', 'rb')
    opp2      = open('NewBase/^IXIC/^IXIC_df'+str(num)+'.pickle', 'rb')
    opp3      = open('NewBase/^DJI/^DJI_df'+str(num)+'.pickle', 'rb')
    opp4      = open('NewBase/GLD/GLD_df'+str(num)+'.pickle', 'rb')
    opp5      = open('NewBase/USO/USO_df'+str(num)+'.pickle', 'rb')
    opp6      = open('NewBase/SPY/SPY_df'+str(num)+'.pickle', 'rb')
    opp7      = open('NewBase/'+key+'/'+key+'_df'+str(num)+'.pickle', 'rb')
    opp8      = open('HLC/'+key+'.pickle', 'rb')
    gspc      = pickle.load(opp)
    ixic      = pickle.load(opp2)
    dji       = pickle.load(opp3)
    gld       = pickle.load(opp4)
    uso       = pickle.load(opp5)
    spy       = pickle.load(opp6)
    df        = pickle.load(opp7)
    hlc       = pickle.load(opp8)
    opp.close()
    opp2.close()
    opp3.close()
    opp4.close()
    opp5.close()
    opp6.close()
    opp7.close()
    opp8.close()
    
    start_date = str(df.index[0])[:10]
    end_date   = str(df.index[-1])[:10]
    closes_df  = hlc['Closes'].loc[start_date:end_date]

    # Also grab the indicator list that contains all the indicators we'll be using from
    # our indicator dataframes since we can't process that many indicators
    try:
        opp9      = open('Pickles/final_lst474'+key+'.pickle','rb')
        final_lst = pickle.load(opp9)
        opp9.close()
    except:
        final_lst = create_indicator_list(key)
        pass

    # Combine the dataframes into a single dataframe, making sure there aren't any
    # duplicate rows which happened curiously a few times during testing.
    comp_lst = [gspc,ixic,dji,gld,uso,spy,df,closes_df]
    for x in range(8):
        comp_lst[x] = comp_lst[x].T.groupby(level=0).first().T
    new_df = pd.concat(comp_lst, axis=1).fillna(method='bfill').fillna(method='ffill')

    # Remove these dates because some dataframes contain data from them but others
    # don't and could affect calculations with up to 390 consecutive incorrect values
    dates = ['2013-08-28', '2013-10-28', '2014-02-12', '2014-02-18', '2014-10-02', 
             '2014-10-06', '2014-10-08', '2014-10-09', '2014-10-13', '2014-10-14', 
             '2014-10-15', '2014-10-20', '2015-01-14', '2015-04-21', '2015-05-18', 
             '2015-06-08', '2015-07-08', '2015-08-20', '2015-08-31', '2015-09-08', 
             '2016-02-08', '2016-03-15', '2016-03-21', '2016-03-22', '2016-04-13', 
             '2016-06-15', '2015-03-30', '2015-05-05', '2014-02-25']
    for each in dates:
        try:
            nums   = new_df.index.get_loc(each)
            new_df = new_df.drop(new_df.index[nums.start:nums.stop])
        except:
            pass
    
    new_df        = new_df.T.groupby(level=0).first().T
    # Create the output indicator name so we can call it from our indicator df
    rets_name     = key+'_rets'+str(out_num)
    # Get a dataframe with just our indicators for training/testing, and then
    # the output column, learning_rets
    result        = new_df[final_lst]
    learning_rets = new_df[rets_name]
    closes_df     = new_df['Closes']

    # Adjust the input/output so inputs/output are in sync. If not, we'd be predicting
    # for for the incorrect returns.
    length        = len(result)
    ret_lst       = [1,2,3,4,5,10,14,16,18,20,25,30,40,50,75,100,125,150,200,250,300]
    result        = result.iloc[:length-ret_lst[out_num]]
    learning_rets = learning_rets.iloc[ret_lst[out_num]:]
    
    # If already have a neural net trained on previous data, net will be true. Then we
    # set our net_var to the already trained net in prep to train further with it.
    net_var = []
    if net == True:
        opp = open('Pickles/net474attrs'+key+'_'+str(out_num)+'.pickle','rb')
        net_var = pickle.load(opp)
        opp.close()
    
    # Set up our test input/output dataframes
    result_test   = result.copy(deep=True)
    testing_rets  = learning_rets.copy(deep=True)
    
    # Leave the last 405 records for testing net w/ test data
    sml = 0
    mid = len(result) - 405
    
    # Convert dataframes to numpy arrays for speed/efficiency purposes
    learning_outputs = learning_rets.iloc[sml:mid]
    testing_outputs  = testing_rets.iloc[mid:mid+400]
    learning_inputs  = result.iloc[sml:mid]
    testing_inputs   = result_test.iloc[mid:mid+400]
    testing_start_dt = testing_inputs.index[0]

    inputs           = learning_inputs.values.T
    outputs          = learning_outputs.values.T
    test_inputs      = testing_inputs.values.T
    test_outputs     = testing_outputs.values.T

    print mid
    return inputs,outputs,test_inputs,test_outputs,key,net_var,testing_start_dt,closes_df

def create_indicator_list(nm_key):
    lst = [nm_key,'^GSPC','^IXIC','^DJI','GLD','USO','SPY']
    lst2 = ['rsi', 'vol', 'sma', 'cci', 'per', 'mom', 'bol', 
            'aro', 'mac', 'mactwo', 'adx', 'kdo', 'rets']
    lst3 = ['rsi', 'vol', 'sma', 'cci', 'per', 'mom', 'bol', 
            'aro', 'mac', 'mactwo', 'adx', 'kdo']
    test = []
    for key in lst:
        for nm in lst2:
            name = key+'_'+nm+'0'
            name2 = key+'_'+nm+'1'
            name3 = key+'_'+nm+'2'
            if (name!=nm_key+'_rets0')or(name2!=nm_key+'_rets1')or(name3!=nm_key+'_rets2'):
                test.append(name)
                test.append(name2)
                test.append(name3)

    for nm in lst3:
        for x in xrange(3,20):
            test.append(nm_key+'_'+nm+str(x))

    opp = open('Pickles/final_lst474'+nm_key+'.pickle','wb')
    pickle.dump(test, opp)
    opp.close()
    return test

print "LOADED"

LOADED


In [13]:
def start_training(inputs, outputs, key, output_num, net=[]):
    """
    If we are using an already created neural network and training futher on it,
    we'll use that to train further, else we create a neural network with 474 
    inputs, 9 hidden neurons, and 1 output.
    """
    # If we don't already have a trained network for further training, create new one
    if net == []:
        net = create_neural_network([474,9,1], 
                    delay_input    = [0,1,2],        
                    delay_internal = [1,2],         
                    delay_output   = [1,2])
    
    # Return our new trained network
    net = train_LM(inputs, outputs, net, verbose=True, iteration_max=10, MSE_stop=1e-10)

    # Dump that network into a pickled file
    opp = open('Pickles/net474attrs'+key+'_'+str(output_num)+'.pickle','wb')
    pickle.dump(net, opp)
    opp.close()
    
    # Return net for testing/graphing
    return net

print "LOADED"

LOADED


In [78]:
def graph_predictions(test_inputs, test_outputs, net, beg, end, inputs=None, outputs=None,):
    """
    Graph the predicted stock return value compared to the actual return value.
    beg is the row your testing, end is the ending row. For example, if 
    beg == 0, and end == 50, we'll look at the predictions for the first 50
    rows of the test inputs.
    """
    if inputs != None:
        previous_inputs  = np.array([]).reshape(0,5)
        previous_outputs = np.array([]).reshape(0,5)
        length = len(inputs[0])
        for prev in (i[length-5:] for i in inputs):
            previous_inputs = np.vstack([previous_inputs, prev])
        previous_outputs = np.vstack([previous_outputs, outputs[length-5:]])
    
        # Create our predictions
        ytest  = NNOut(test_inputs, net, P0=previous_inputs, Y0=previous_outputs[0])
    else:
        ytest  = NNOut(test_inputs, net)
    
    # On how much data will we graph
    total  = end - beg
    
    # Graph our predicted vs actual output
    fig = plt.figure()
    plt.plot(range(total), ytest[beg:end], 'b-', label='test')
    plt.plot(range(total), test_outputs[beg:end], 'r', label='real')
    fig.suptitle('Predicted VS Actual Output', fontsize=20)
    plt.xlabel('Timestamp', fontsize=18)
    plt.ylabel('Return Value', fontsize=16)
    blue_patch = mpatches.Patch(color='blue', label='Predicted Output')
    red_patch  = mpatches.Patch(color='red', label='Actual Output')
    plt.legend(handles=[red_patch, blue_patch])
    plt.show()
    return

def get_prediction_results(test_inputs, test_outputs, net, test_dt, 
                           closes_df, key, inputs=None, outputs=None):
    """
    Here we'll take our trained network and test our predictions and for each
    interval make a choice to either buy, short, or stay. We'll then take those
    decisions and test them against the actual results in return values and either
    add or subtract to our profitability variable. Print the results after this
    is done.
    """
    guess_list = []
    dff  = pd.DataFrame(columns=['cls','out','chg','gue','res','rgue','res2'],
                        index=[range(len(test_outputs))])
    rand = {True:'buy',False:'short'}
    result,result2,count = 0,0,0
    correct,correct2,incorrect,incorrect2 = 0.,0.,0.,0.
    #closes = closes_df.loc[test_dt:].iloc[:20098].values.tolist()
    closes = closes_df.loc[test_dt:].iloc[:len(closes_df)-3406].values.tolist()
    
    # Make our predictions using our trained network and test inputs, with optional
    # addition of using the previous 5 inputs/outputs.
    if inputs != None:
        previous_inputs  = np.array([]).reshape(0,5)
        previous_outputs = np.array([]).reshape(0,5)
        length = len(inputs[0])
        for prev in (i[length-5:] for i in inputs):
            previous_inputs = np.vstack([previous_inputs, prev])
        previous_outputs = np.vstack([previous_outputs, outputs[length-5:]])
        ytest  = NNOut(test_inputs, net, P0=previous_inputs, Y0=previous_outputs[0])
    else:
        ytest  = NNOut(test_inputs, net)
    
    # For each return guess value, if its expected to be greater, we choose buy
    # if less, we choose short, if 0, we choose stay
    for return_guess in ytest:
        if return_guess > 0:
            guess_list.append('buy')
        elif return_guess < 0:
            guess_list.append('short')
        else:
            guess_list.append('stay')
    
    # If we choose the right choice, we gain that amount of price difference
    # if not, we lose that price difference. If the actual output is 0, regardless
    # of what we choose, we stay the same.
    for o,g,c in zip(test_outputs, guess_list, closes):
        price_change = 1000 * o * c
        if g == 'buy':
            if o > 0:
                result += price_change
                correct += 1.
            else:
                result -= abs(price_change)
                if o < 0:
                    incorrect += 1
        elif g == 'short':
            if o < 0:
                result += abs(price_change)
                correct += 1
            else:
                result -= price_change
                if o > 0:
                    incorrect += 1

        rg = rand[bool(random.getrandbits(1))]
        if rg == 'buy':
            if o > 0:
                result2 += price_change
                correct2 += 1.
            else:
                result2 -= abs(price_change)
                if o < 0:
                    incorrect2 += 1.
        elif rg == 'short':
            if o < 0:
                result2 += abs(price_change)
                correct2 += 1
            else:
                result2 -= price_change
                if o > 0:
                    incorrect2 += 1

        dff.iloc[count] = [c,o,price_change,g,result,rg,result2]
        count += 1
                
    if result >= 0:
        print 'Success, profitable:', result, '% correct:', correct/(correct+incorrect)
    else:
        print 'Fail, unprofitable:', result, '% correct:', correct/(correct+incorrect)
    if result2 >= 0:
        print 'Random success, profitable:', result2, '% correct:', correct2/(correct2+incorrect2)
    else:
        print 'Random fail, unprofitable:', result2, '% correct:', correct2/(correct2+incorrect2)
    return dff

print "LOADED"

LOADED


In [1]:
# The first parameter is the piece of indicators to work on, for example 0, is the oldest data,
# while 14 is the newest data. For testing you can train a network with a single piece or use
# all of them, simply adding 1 after each testing round. It will automatically utilize 
# the trained network by setting net=True after you've trained the network for at least a 
# single piece. The second parameter is the stock indicator symbol that you are training for. The
# third parameter is the return length you're predicting for, 0-31, with 0 being, 1 minute ahead,
# up to 31 being 125 days into the future. 4th optional parameter is net=True or False, false, if
# you haven't trained on at least one piece of the indicator dataframe for that company, true if
# you have.
inputs,outputs,test_inputs,test_outputs,key,net,testing_dt,closes_df=create_learntest(0,'SIRI',0)
#inputs,outputs,test_inputs,test_outputs,key,net,testing_dt,closes_df=create_learntest(1,'SIRI',0,net=True)

In [73]:
# Start training using the inputs, outputs, stock symbol, the return period your predicting for,
# and the net. If it's your first time training and you don't have a net yet, net is passed in as
# [] from the create_learntest() function and it automatically sets up a net for you in this 
# function when you pass that.
net = start_training(inputs, outputs, key, 0, net=net)

In [82]:
# This will graph 100 stock minutes of test output for you
graph_predictions(test_inputs, test_outputs, net, 10, 110)

In [84]:
# This will return a dataframe containing your guess, the outcomes, and profitability
dff = get_prediction_results(test_inputs, test_outputs, net, testing_dt, closes_df, key)